In [ ]:
# !pip install deepchem -q
!pip install scikit-learn==1.0.2
!pip install XlsxWriter

#import the necessary libraries
import os
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
import math
from sklearn.ensemble import RandomForestRegressor
from numpy import inf
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from pandas import DataFrame
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.feature_selection import VarianceThreshold

from torch.utils.data import TensorDataset

from tqdm import tqdm
from math import sqrt
from rdkit.Chem import rdMolDescriptors
import random

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

!pip install -U geometric-smote
from gsmote import GeometricSMOTE
#from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_excel("./866-reaction-mean-std-cv-1-20-encoding-vector.xlsx")
df.drop(columns=['Sl. No.'], inplace=True)
df=df.iloc[:692]
descriptor_data=df.drop(columns=['label'])

In [ ]:
#------------Stratified data splitting into train, validation and test sets before oversampling-----------
#beware of data leakage always; split first always
random_state=51
y_class=[]
boundary=53
for i in descriptor_data['Yield']:
  if i<=boundary:
    y_class.append(0)
  elif i>boundary:
    y_class.append(1)
print(len(y_class))
print('Number of reactions below boundary:', y_class.count(0), 'and above boundary:', y_class.count(1))

In [ ]:
#preparing train_val and test set
train_ , test_df = train_test_split(descriptor_data, test_size=0.20, random_state=random_state, stratify=y_class) #preparing train_val and test set
y_class2=[]
boundary=53
for i in train_['Yield']:
  if i<=boundary:
    y_class2.append(0)
  elif i>boundary:
    y_class2.append(1)
print('Number of reactions below boundary:', y_class2.count(0), 'and above boundary:', y_class2.count(1))
train_df, valid_df = train_test_split(train_, test_size=0.125, random_state=random_state, stratify=y_class2) #preparing train and val set
print('Data shapes:', train_df.shape, valid_df.shape, test_df.shape)

print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

In [ ]:
def smote(train_df,  smote_type, boundary):
    #selection of type of smote
    if smote_type=='regular':
      sm=SMOTE(random_state=42)
      print(sm)
    if smote_type=='borderline-1':
      sm=BorderlineSMOTE(kind=smote_type, random_state=42)
      print(sm)
    elif smote_type=='borderline-2':
      sm=BorderlineSMOTE(kind=smote_type, random_state=42)
      print(sm)
    elif smote_type=='svm':
      sm=SVMSMOTE(random_state=42)
      print(sm)
    elif smote_type=='GeometricSMOTE':
      sm=GeometricSMOTE(random_state=42,n_jobs=1,deformation_factor=0.2,truncation_factor=0.2,k_neighbors=6,selection_strategy='minority')
      print(sm)
    elif smote_type=='random':
      sm=RandomOverSampler()
      print(sm)
    #adding the label column with the condition
    label=[]
    for i in train_df['Yield']:
        if i <=boundary:
            label.append(0)
        else:
            label.append(1)
    train_df['label']=label
    X=train_df.iloc[:, :-1]
    y=train_df.iloc[:, -1]
    X_aug, y_aug = sm.fit_resample(X, y)
    return X_aug

In [ ]:
train_df_aug=smote(train_df, smote_type='regular', boundary=53)

In [ ]:
train_df_aug.to_csv('smote-regular.csv', index = False)